#### Libraries and Data

In [6]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [9]:
mnist_dataset, mnist_info=tfds.load(name='mnist', with_info=True, as_supervised=True)

#### Preprocessing

In [10]:
mnist_train=mnist_dataset['train']
mnist_test=mnist_dataset['test']

num_validation_samples= 0.1 *mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples, tf.int64)

num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples, tf.int64)

def scale(image,label):
    image=tf.cast(image, tf.float32)
    image /= 255.
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)
test_data=mnist_test.map(scale)

BUFFER_SIZE=10000
#we can't shuffle all dataset all at once if it's big enough
shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

#batch-size=1: SGD
BATCH_SIZE=100
train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_samples)

validation_inputs, validation_targets=next(iter(validation_data))

#### Model outline

In [16]:
input_size=784
output_size=10
hidden_layer_size=100 #assuming all hidden layers are equa

model=tf.keras.Sequential([
                          tf.keras.layers.Flatten(input_shape=(28,28,1)),
                          tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                          tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                          tf.keras.layers.Dense(output_size, activation='softmax')                    
                          ])

#### Optimizer and loss function

In [17]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#### Training

In [18]:
NUM_EPOCHS=5

model.fit(train_data, epochs= NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 3s - loss: 0.3353 - accuracy: 0.9053 - val_loss: 0.1644 - val_accuracy: 0.9512 - 3s/epoch - 6ms/step
Epoch 2/5
540/540 - 3s - loss: 0.1385 - accuracy: 0.9587 - val_loss: 0.1179 - val_accuracy: 0.9620 - 3s/epoch - 5ms/step
Epoch 3/5
540/540 - 2s - loss: 0.0982 - accuracy: 0.9702 - val_loss: 0.0929 - val_accuracy: 0.9727 - 2s/epoch - 5ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0748 - accuracy: 0.9769 - val_loss: 0.0740 - val_accuracy: 0.9785 - 2s/epoch - 4ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0591 - accuracy: 0.9822 - val_loss: 0.0642 - val_accuracy: 0.9808 - 2s/epoch - 4ms/step


#### Test

In [20]:
test_loss, test_accuracy=model.evaluate(test_data)
print('Test loss: {0:.2f}, Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 [==============================] - 0s 230ms/step - loss: 0.0766 - accuracy: 0.9756
Test loss: 0.08, Test accuracy: 97.56%


Test accuracy is close to validation accuracy so there is no overfitting